# DiploDatos - Mentoria Financial DataSet
### Grupo 23

**Integrantes:** Franco Amilcar Genolet, Federico Bossack, Marilina Trevisan, Catalina Chavez.

### Librerías

In [17]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno

from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay, classification_report
from sklearn.impute import KNNImputer
from sklearn.metrics import precision_score, accuracy_score, recall_score

from sklearn.tree import DecisionTreeClassifier

import warnings
warnings.filterwarnings("ignore") 

### Importación del dataset

In [18]:
df = pd.read_csv("https://raw.githubusercontent.com/francogeno/Diplo-Data-Science/Mentoria/Mentoriadefinitive_financial_w_pca.csv")

### Exporación y análisis de los datos

In [19]:
df.head()

,Class,AnualPriceVariation,Revenue,Cost of Revenue,Operating Income,Interest Expense,Net Income,EPS,EBITDA,Total current liabilities,...,Año_2014,Año_2015,Año_2016,Año_2017,Año_2018,pca_col_1,pca_col_2,pca_col_3,pca_col_4,pca_col_5
0,0,-9.323276,1.583704e+09,1.007561e+09,1.723658e+08,4.300989e+07,1.048335e+08,4.1900,2.842308e+08,7.526881e+08,...,1,0,0,0,0,1.003036,-2.304296,-1.007372,-0.210671,-0.503676
1,0,-25.512193,3.734148e+09,2.805625e+09,1.345959e+08,1.214869e+07,1.358227e+08,0.2396,2.456858e+08,1.639537e+09,...,1,0,0,0,0,1.378006,-0.287206,-1.260708,0.170942,-0.395603
2,1,33.118297,1.583704e+09,1.007561e+09,1.723658e+08,4.300989e+07,1.048335e+08,1.4700,2.842308e+08,7.526881e+08,...,1,0,0,0,0,1.599254,-0.552765,-1.516758,-0.068516,-0.500159
3,1,2.752291,1.583704e+09,1.007561e+09,7.372520e+08,4.300989e+07,2.494140e+08,4.6000,1.078550e+09,2.507452e+09,...,1,0,0,0,0,4.228032,-2.091455,-0.690725,-0.089458,-0.189818
4,1,12.897715,1.583704e+09,1.007561e+09,1.723658e+08,3.024000e+08,1.048335e+08,2.9000,2.842308e+08,5.423500e+09,...,1,0,0,0,0,3.727582,-0.148934,1.423555,-0.208093,-0.306327


In [20]:
df.shape

(22077, 37)

In [21]:
def status(data):
    data2=data
    # total de rows
    tot_rows=len(data2)
    # total de nan
    d2=data2.isnull().sum().reset_index()
    d2.columns=['variable', 'q_nan']
    # percentage of nan
    d2[['p_nan']]=d2[['q_nan']]/tot_rows
    # num of zeros
    d2['q_zeros']=(data2==0).sum().values
    # perc of zeros
    d2['p_zeros']=d2[['q_zeros']]/tot_rows
    # total unique values
    d2['unique']=data2.nunique().values
    # get data types per column
    d2['type']=[str(x) for x in data2.dtypes.values]
    return(d2)
status(df)

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
0,Class,0,0.0,9918,0.449246,2,int64
1,AnualPriceVariation,0,0.0,25,0.001132,21940,float64
2,Revenue,0,0.0,0,0.000000,17908,float64
3,Cost of Revenue,0,0.0,0,0.000000,14137,float64
4,Operating Income,0,0.0,32,0.001449,17367,float64
5,Interest Expense,0,0.0,0,0.000000,10311,float64
6,Net Income,0,0.0,43,0.001948,16735,float64
7,EPS,0,0.0,91,0.004122,2453,float64
8,EBITDA,0,0.0,9,0.000408,17064,float64
9,Total current liabilities,0,0.0,0,0.000000,16475,float64


Division del dataset en variables predictoras y variables a predecir:
- Para este notebook, se preparará un modelo de clasificación, por lo que la variable a predecir será *Class*

In [22]:
X = df.iloc[:,2:32]
print(X.columns.values)
y = df["Class"]

['Revenue' 'Cost of Revenue' 'Operating Income' 'Interest Expense'
 'Net Income' 'EPS' 'EBITDA' 'Total current liabilities'
 'Total liabilities' 'Operating Cash Flow' 'returnOnAssets'
 'Net Income per Share' 'Market Cap' 'EPS Diluted Growth'
 'Basic Materials' 'Communication Services' 'Consumer Cyclical'
 'Consumer Defensive' 'Energy' 'Financial Services' 'Healthcare'
 'Industrials' 'Real Estate' 'Technology' 'Utilities' 'Año_2014'
 'Año_2015' 'Año_2016' 'Año_2017' 'Año_2018']


In [23]:
X.columns.values
# X = X.drop(['Año_2014', 'Año_2015', 'Año_2016', 'Año_2017', 'Año_2018'], axis = 1)

array(['Revenue', 'Cost of Revenue', 'Operating Income',
       'Interest Expense', 'Net Income', 'EPS', 'EBITDA',
       'Total current liabilities', 'Total liabilities',
       'Operating Cash Flow', 'returnOnAssets', 'Net Income per Share',
       'Market Cap', 'EPS Diluted Growth', 'Basic Materials',
       'Communication Services', 'Consumer Cyclical',
       'Consumer Defensive', 'Energy', 'Financial Services', 'Healthcare',
       'Industrials', 'Real Estate', 'Technology', 'Utilities',
       'Año_2014', 'Año_2015', 'Año_2016', 'Año_2017', 'Año_2018'],
      dtype=object)

Se comprueba con otro codigo que el año influye en la precision del modelo. Razon por la cual, se comenta la celda de "drop".

In [24]:
# Escalamos
x_names = X.columns
scaler = MinMaxScaler(feature_range=(0, 1), copy=True)
X_orig = X.copy(deep=True)
X = scaler.fit_transform(X_orig)
X = pd.DataFrame(X, columns=x_names)
X

,Revenue,Cost of Revenue,Operating Income,Interest Expense,Net Income,EPS,EBITDA,Total current liabilities,Total liabilities,Operating Cash Flow,...,Healthcare,Industrials,Real Estate,Technology,Utilities,Año_2014,Año_2015,Año_2016,Año_2017,Año_2018
0,0.109392,0.128620,0.139720,0.138295,0.177249,0.916667,0.121655,0.097122,0.102753,0.117878,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.257937,0.358153,0.120357,0.039063,0.199533,0.546779,0.108251,0.211961,0.098357,0.242046,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.109392,0.128620,0.139720,0.138295,0.177249,0.661985,0.121655,0.097122,0.102753,0.117878,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.109392,0.128620,0.429303,0.138295,0.281215,0.955056,0.397882,0.324349,0.387743,0.315202,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.109392,0.128620,0.139720,0.972347,0.177249,0.795880,0.121655,0.701952,0.648872,0.117878,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22072,0.109392,0.128620,0.049433,0.035558,0.092002,0.516854,0.021506,0.007005,0.009005,0.019598,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
22073,0.000034,0.128620,0.046604,0.138295,0.095271,0.438202,0.019692,0.097122,0.000124,0.018574,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
22074,0.003787,0.004671,0.056288,0.003984,0.106912,0.573034,0.026312,0.004723,0.001489,0.021294,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
22075,0.109392,0.128620,0.042823,0.138295,0.089894,0.507491,0.017095,0.097122,0.102753,0.017575,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


### Entrenamiento y Test

In [25]:
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state = 8)

In [27]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((17661, 30), (17661,), (4416, 30), (4416,))

### Modelo Baseline

In [28]:
clfs = [DecisionTreeClassifier()]
names = ['Árbol de decisión']

trained_models = []
accuracy_models = []
for clf,name in zip(clfs,names):
    print(name)
    clf.fit(x_train, y_train)
    train_predictions = clf.predict(x_train)
    accuracy = accuracy_score(y_train, train_predictions)
    print(f"Accuracy train {name}: %.2f%%" % (accuracy * 100.0))

    train_predictions = clf.predict(x_test)
    accuracy = accuracy_score(y_test, train_predictions)
    print("Accuracy test {name}: %.2f%%" % (accuracy * 100.0))
    # plot_confusion_matrix(clf,x_test,y_test)
    trained_models.append(clf)
    accuracy_models.append(accuracy*100)

Árbol de decisión
Accuracy train Árbol de decisión: 99.47%
Accuracy test {name}: 63.22%


### Algún tipo de ajuste de hiperparámetros de los modelos elegidos

In [29]:
grid = GridSearchCV(DecisionTreeClassifier(),
 {'criterion':('gini','entropy'),
 'splitter':("best","random"),
 'max_depth':(10,20,100)})
grid.fit(x_train,y_train)
grid.best_params_

{'criterion': 'gini', 'max_depth': 10, 'splitter': 'random'}

In [30]:
clf = DecisionTreeClassifier(**grid.best_params_).fit(x_train, y_train)

train_predictions = clf.predict(x_train)
accuracy = accuracy_score(y_train, train_predictions)
print("Accuracy train Arbol de decisión: %.2f%%" % (accuracy * 100.0))

test_predictions = clf.predict(x_test)
accuracy = accuracy_score(y_test, test_predictions)
print("Accuracy test Arbol de decisión: %.2f%%" % (accuracy * 100.0))

print(classification_report(y_test,clf.predict(x_test)))

Accuracy train Arbol de decisión: 73.38%
Accuracy test Arbol de decisión: 68.18%
              precision    recall  f1-score   support

           0       0.64      0.66      0.65      1959
           1       0.72      0.70      0.71      2457

    accuracy                           0.68      4416
   macro avg       0.68      0.68      0.68      4416
weighted avg       0.68      0.68      0.68      4416



In [31]:
FOLDS=5
cv = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=10)

x_train2 = np.array(x_train)
y_train2 = np.array(y_train)


for clfi, name in zip([DecisionTreeClassifier(**grid.best_params_)],names):
    print(clfi)
    avg_accuracy = 0
    for fold, (train_idx, val_idx) in enumerate(cv.split(x_train2, y_train2)):
        xi, yi = x_train2[train_idx], y_train2[train_idx]
        x_valid, y_valid = x_train2[val_idx], y_train2[val_idx]
        clfi = clfi.fit(xi, yi)

        test_predictions = clfi.predict(x_valid)
        accuracy = accuracy_score(y_valid, test_predictions)
        avg_accuracy +=accuracy
        print(f"Precisión test fold {fold}: {accuracy * 100.0 :.2f}" % ())

    avg_accuracy /= FOLDS
    print(f'Avg. accuracy = {avg_accuracy * 100}')

DecisionTreeClassifier(max_depth=10, splitter='random')
Precisión test fold 0: 69.57
Precisión test fold 1: 70.02
Precisión test fold 2: 69.17
Precisión test fold 3: 69.31
Precisión test fold 4: 68.86
Avg. accuracy = 69.38450891272996


### Validamos ahora varios modelos para obtener mayor Accuracy

### Primero validaremos con varios modelos para verificar cual puede ser que obtenga un mayor Accuracy

In [32]:
from sklearn.model_selection import cross_val_score, cross_validate, KFold, StratifiedKFold, LeaveOneOut, LeavePOut
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, roc_auc_score, mean_squared_error, r2_score, confusion_matrix, classification_report, ConfusionMatrixDisplay

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC

clfs =  [DecisionTreeClassifier(),
        RandomForestClassifier(),
        LogisticRegression(),
        MLPClassifier(),
        XGBClassifier(),
        GaussianNB(),
        LinearSVC()]

names = ['Arbol de decisión',
        'Random Forest',
        'Regresión Logística',
        'Perceptrón multicapa',
        'XGBoost',
        'Naive Bayes',
        'SVM']

trained_models = []
accuracy_models = []
for clf, name in zip(clfs, names):
    print(name)
    clf.fit(x_train, y_train)
    
    train_predictions = clf.predict(x_train)
    accuracy = accuracy_score(y_train, train_predictions)
    print(f"Accuracy train {name}: %.2f%%" % (accuracy * 100.0))

    test_predictions = clf.predict(x_test)
    accuracy = accuracy_score(y_test, test_predictions)
    print(f"Accuracy test {name}: %.2f%%" % (accuracy * 100.0))
    trained_models.append(clf)    
    accuracy_models.append(accuracy)

Arbol de decisión
Accuracy train Arbol de decisión: 99.47%
Accuracy test Arbol de decisión: 62.70%
Random Forest
Accuracy train Random Forest: 99.47%
Accuracy test Random Forest: 69.32%
Regresión Logística
Accuracy train Regresión Logística: 68.90%
Accuracy test Regresión Logística: 67.48%
Perceptrón multicapa
Accuracy train Perceptrón multicapa: 73.38%
Accuracy test Perceptrón multicapa: 68.30%
XGBoost
Accuracy train XGBoost: 86.14%
Accuracy test XGBoost: 68.16%
Naive Bayes
Accuracy train Naive Bayes: 66.63%
Accuracy test Naive Bayes: 63.79%
SVM
Accuracy train SVM: 68.77%
Accuracy test SVM: 67.48%


In [33]:
models = pd.DataFrame({'Model':names, 'Score':accuracy_models})
models.sort_values(by='Score', ascending=False)

,Model,Score
1,Random Forest,0.693161
3,Perceptrón multicapa,0.682971
4,XGBoost,0.681612
2,Regresión Logística,0.674819
6,SVM,0.674819
5,Naive Bayes,0.637908
0,Arbol de decisión,0.627038


#### En esta validacion de por todos los modelos, se puede observar que el que mejor comportamiento se obtuvo fue el de XGBoost, ya que obtiene un balance entre el tran y el test

#### Ahora validaremos cuales serian los mejores parametros para el modelo de XGBoost.

In [34]:
from xgboost import XGBClassifier

params={
 "learning_rate"    : [0.05, 0.10, 0.1] ,
 "max_depth"        : [ 3, 4, 5],
 "min_child_weight" : [ 1, 3, 5],
 "gamma"            : [ 0.0, 0.1, 0.2]   
}


In [35]:
classifier=XGBClassifier()

In [36]:
grid = GridSearchCV(classifier,params)

In [ ]:
# grid.fit(x_train,y_train)

In [ ]:
# grid.best_params_

In [ ]:
# clfxboost = XGBClassifier(**grid.best_params_).fit(x_train, y_train)

# train_predictions = clfxboost.predict(x_train)
# accuracy = accuracy_score(y_train, train_predictions)
# print("Accuracy train Arbol de decisión: %.2f%%" % (accuracy * 100.0))

# test_predictions = clfxboost.predict(x_test)
# accuracy = accuracy_score(y_test, test_predictions)
# print("Accuracy test Arbol de decisión: %.2f%%" % (accuracy * 100.0))
# plt.show()

In [ ]:
# print(classification_report(y_test, test_predictions))

In [ ]:
# print(classification_report(y_test, test_predictions))
# from sklearn.metrics import confusion_matrix,roc_curve

# cm = confusion_matrix(y_test, test_predictions)
# f = sns.heatmap(cm, annot=True, fmt='d')

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test,clfxboost.predict_proba(x_test)[:,1])
plt.plot(fpr, tpr, label='ROC curve')
plt.plot([0, 1], [0, 1], 'k--', label='Random guess')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.xlim([-0.02, 1])
plt.ylim([0, 1.02])
plt.legend(loc="lower right")

In [ ]:
roc_auc_score(y_test, test_predictions)

#### Debido a que el GridsearchCV para este este modelo se tarda, validaremos tambien por RandomSearchCV para verificar si se obtienen mejores o mismos resultados

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

random_search = RandomizedSearchCV(classifier,param_distributions=params,n_iter=30,scoring='roc_auc',n_jobs=-1,cv=5,verbose=3)
random_search.fit(x_train,y_train)

In [ ]:
random_search.best_params_

In [ ]:
clfrandom_search = XGBClassifier(**random_search.best_params_).fit(x_train, y_train)

train_predictions = clfrandom_search.predict(x_train)
accuracy = accuracy_score(y_train, train_predictions)
print("Accuracy train Arbol de decisión: %.2f%%" % (accuracy * 100.0))

test_predictions = clfrandom_search.predict(x_test)
accuracy = accuracy_score(y_test, test_predictions)
print("Accuracy test Arbol de decisión: %.2f%%" % (accuracy * 100.0))
plt.show()


In [ ]:
print(classification_report(y_test, test_predictions))

In [ ]:
from sklearn.metrics import confusion_matrix,roc_curve

cm = confusion_matrix(y_test, test_predictions)
f = sns.heatmap(cm, annot=True, fmt='d')

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test,clfrandom_search.predict_proba(x_test)[:,1])
plt.plot(fpr, tpr, label='ROC curve')
plt.plot([0, 1], [0, 1], 'k--', label='Random guess')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.xlim([-0.02, 1])
plt.ylim([0, 1.02])
plt.legend(loc="lower right")

In [ ]:
roc_auc_score(y_test, test_predictions)

#### Por lo analizado anteriormente, los mejores parametros se obtuvieron con el GridsearchCV aun cuando su procesamiento sea tardio, ahora validaremos un modelo diferente para verificar si se consigo una payor precision.

In [ ]:
param_grid = {
             'max_depth'   : [3, 10, 20],
             'min_samples_leaf': [0.05, 0.1],
             'max_features': ['sqrt', 'log2'],
             'ccp_alpha': [0, 0.01]
            }

model = RandomForestClassifier(
            n_estimators = 100,
            oob_score    = True,
            n_jobs       = -1,
            random_state = 123
        )
grid = GridSearchCV(
        estimator  = model,
        param_grid = param_grid,
        scoring    = 'roc_auc',
        cv         = 5, 
        refit      = False,
        verbose    = 0,
        return_train_score = True
       )

In [ ]:
grid.fit(x_train,y_train)

In [ ]:
grid.best_params_

In [ ]:
clfrandomForest = RandomForestClassifier(**grid.best_params_).fit(x_train, y_train)

train_predictions = clfrandomForest.predict(x_train)
accuracy = accuracy_score(y_train, train_predictions)
print("Accuracy train Arbol de decisión: %.2f%%" % (accuracy * 100.0))

test_predictions = clfrandomForest.predict(x_test)
accuracy = accuracy_score(y_test, test_predictions)
print("Accuracy test Arbol de decisión: %.2f%%" % (accuracy * 100.0))
plt.show()

In [ ]:
print(classification_report(y_test, test_predictions))

In [ ]:
from sklearn.metrics import confusion_matrix,roc_curve

cm = confusion_matrix(y_test, test_predictions)
f = sns.heatmap(cm, annot=True, fmt='d')

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test,clfrandomForest.predict_proba(x_test)[:,1])
plt.plot(fpr, tpr, label='ROC curve')
plt.plot([0, 1], [0, 1], 'k--', label='Random guess')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.xlim([-0.02, 1])
plt.ylim([0, 1.02])
plt.legend(loc="lower right")

In [ ]:
roc_auc_score(y_test, test_predictions)

## En conclusion, el mejor modelo es el Random Forset con una precision para el conjunto de validacion de 69,32%